In [ ]:
# Azure Route Table JSON to Excel converter

# --- Colab Setup ---
!pip install pandas openpyxl

import json, os, re
import pandas as pd
from google.colab import files
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

# --- Region Formatter (Universal) ---
def format_location(loc: str) -> str:
    if not loc:
        return ""
    loc_lower = loc.lower()
    region_map = {  # abbreviated list; see full universal map in previous step
        "australiaeast":"Australia East","australiasoutheast":"Australia Southeast",
        "eastus":"East US","eastus2":"East US 2","westeurope":"West Europe",
        "northeurope":"North Europe","uksouth":"UK South","ukwest":"UK West",
        "southeastasia":"Southeast Asia","centralus":"Central US","southcentralus":"South Central US",
        "westus":"West US","westus2":"West US 2","canadacentral":"Canada Central",
        "canadaeast":"Canada East","brazilsouth":"Brazil South","japaneast":"Japan East",
        "japanwest":"Japan West","francecentral":"France Central","germanywestcentral":"Germany West Central"
    }
    if loc_lower in region_map: return region_map[loc_lower]
    return re.sub(r'([a-z])([A-Z0-9])', r'\1 \2', loc_lower.title()).replace("-", " ").title()


# --- Upload JSON ---
print("📤 Upload your Route Table JSON file:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# --- Parse JSON ---
with open(filename,'r') as f: data=json.load(f)
rt_name=data.get("name","Unknown Route Table")
id_path=data.get("id","")
subscription_id=id_path.split("/")[2] if id_path else ""
resource_group=id_path.split("/resourceGroups/")[1].split("/")[0] if "/resourceGroups/" in id_path else ""
metadata={
 "Resource group":resource_group,
 "Location":format_location(data.get("location","")),
 "Subscription ID":subscription_id
}

# --- ROUTES ---
routes=[]
for r in data["properties"].get("routes",[]):
  p=r.get("properties",{})
  routes.append([r.get("name",""),p.get("addressPrefix",""),p.get("nextHopType",""),p.get("nextHopIpAddress","")])
df_routes=pd.DataFrame(routes,columns=["Name","Address Prefix","Next Hop Type","Next Hop IP Address"]).astype(str)

# --- SUBNETS ---
subnets=[]
for s in data["properties"].get("subnets",[]):
  sid=s["id"]
  name=sid.split("/")[-1]
  vnet=sid.split("/virtualNetworks/")[1].split("/subnets/")[0] if "/virtualNetworks/" in sid else ""
  p=s.get("properties",{})
  addr=p.get("addressPrefix","")
  nsg=p.get("networkSecurityGroup",{}).get("id","")
  nsg=nsg.split("/")[-1] if nsg else ""
  subnets.append([name,addr,vnet,nsg])
df_sub=pd.DataFrame(subnets,columns=["Name","Address Range","Virtual Network","Security Group"]).astype(str)

# --- Excel ---
wb=Workbook(); ws=wb.active; ws.title="ROUTE_TABLE"
bold, title=Font(bold=True), Font(bold=True,size=14)
fill_hdr=PatternFill("solid","D9E1F2"); fill_title=PatternFill("solid","BDD7EE")
align_center=Alignment(horizontal="center")
border=Border(left=Side(style="thin"),right=Side(style="thin"),top=Side(style="thin"),bottom=Side(style="thin"))

ws.append([rt_name]); ws.merge_cells(start_row=1,start_column=1,end_row=1,end_column=4)
ws["A1"].font=title; ws["A1"].fill=fill_title; ws["A1"].alignment=align_center
ws.append([""])
for k,v in metadata.items(): ws.append([k,v]); ws[f"A{ws.max_row}"].font=bold
ws.append([""])
ws.append(["ROUTES"])
ws.append(df_routes.columns.tolist())
for i in range(1,5):
  cell=ws[f"{get_column_letter(i)}{ws.max_row}"]; cell.font=bold; cell.fill=fill_hdr
for r in df_routes.itertuples(index=False): ws.append(list(r))
ws.append([""])
ws.append(["SUBNETS"])
ws.append(df_sub.columns.tolist())
for i in range(1,5):
  cell=ws[f"{get_column_letter(i)}{ws.max_row}"]; cell.font=bold; cell.fill=fill_hdr
for r in df_sub.itertuples(index=False): ws.append(list(r))
for row in ws.iter_rows():
  for c in row:
    if c.value: c.border=border
for col in ws.columns:
  ws.column_dimensions[get_column_letter(col[0].column)].width=max(len(str(c.value)) for c in col if c.value)+3
out=os.path.splitext(filename)[0]+".xlsx"; wb.save(out)
files.download(out)
